In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import json, os, csv, shutil
from glob import glob
import matplotlib.pyplot as plt

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Lambda, Activation, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop, SGD
from keras.utils import np_utils

import h5py
import cv2

In [5]:
path = 'data/'

In [6]:
batch_size = 32

# Data preprocessing

In [20]:
dir_names = os.listdir(os.path.join(path, 'train/'))
dir_names

['NoF', 'OTHER', 'ALB', 'DOL', 'BET', 'LAG', 'YFT', 'SHARK']

In [21]:
dir_dict = {}

for i in range(len(dir_names)):
    dir_dict[dir_names[i]] = i

In [22]:
dir_dict

{'ALB': 2,
 'BET': 4,
 'DOL': 3,
 'LAG': 5,
 'NoF': 0,
 'OTHER': 1,
 'SHARK': 7,
 'YFT': 6}

In [24]:
X_train = []
y_train = []

In [25]:
for folder in os.listdir(path + 'train/'):
    for file in os.listdir(path + 'train/' + folder + '/'):
        if os.path.isfile(path + 'train/' + folder + '/' + file):
            
            input_img = cv2.imread(path + 'train/' + folder + '/' + file)
            input_img = cv2.resize(input_img, (224, 224))
            X_train.append(input_img)
            
            y_cat = np_utils.to_categorical(dir_dict[folder], 8) # 8 represent number of class
            y_train.append(y_cat)

In [27]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [28]:
print(X_train.shape)
print(y_train.shape)

(3025, 224, 224, 3)
(3025, 8)


In [39]:
y_train[466]

array([0., 1., 0., 0., 0., 0., 0., 0.])

In [30]:
X_valid = []
y_valid = []

In [31]:
for folder in os.listdir(path + 'valid/'):
    for file in os.listdir(path + 'valid/' + folder + '/'):
        if os.path.isfile(path + 'valid/' + folder + '/' + file):
            
            input_img = cv2.imread(path + 'valid/' + folder + '/' + file)
            input_img = cv2.resize(input_img, (224, 224))
            X_valid.append(input_img)
            
            y_cat = np_utils.to_categorical(dir_dict[folder], 8) # 8 represent number of class
            y_valid.append(y_cat)

In [32]:
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

In [33]:
print(X_valid.shape)
print(y_valid.shape)

(752, 224, 224, 3)
(752, 8)


In [35]:
y_valid[0]

array([1., 0., 0., 0., 0., 0., 0., 0.])

# NN

In [41]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(32, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(8, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ["accuracy"])

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 200704)            0         
__________

In [43]:
model.fit(X_train, y_train, epochs = 1, batch_size = batch_size, validation_data = [X_valid, y_valid])

Train on 3025 samples, validate on 752 samples
Epoch 1/1
  64/3025 [..............................] - ETA: 1:17:31 - loss: 9.4753 - acc: 0.2812 

KeyboardInterrupt: 